# DNA Sequence Classification

DNA sequences can be classified in different ways. This demo shows how to build a system with Milvus 2.0 and postgres to determine gene families for DNA sequences and compare similarity between different organisms. It uses k-mer and CountVectorizer to extract features and get embeddings for dna sequences.

## Data

Sample data is downloaded from [Kaggle](https://www.kaggle.com/nageshsingh/dna-sequence-dataset?select=human.txt). This demo uses 6882 DNA sequences for 3 organisms: human (4380), chimpanzee (1682), dog (820), each of which has lines of DNA sequences with corresponding classes in a text file.

     - data:
        - human_data.txt
        - chimp_data.txt
        - dog_data.txt

Each text file consists of a head line and lines of sequence (a sequence consisting of bases [A, C, G, T]) with its class (an integer from 0 to 6). Sample:
                       
     sequence class
     GCTGCTGCCCCAGCACCAGGTGTCCGCGTACTGA	6
     CACCGGCCCTCCAGGGTCCAGCTGGTGCCCCAGGACACCATGACCAGCAGGGCCTAA	0

Our DNA sequences are classified into different gene families by class. A gene family is a group of related genes sharing a common ancestor. Members of one gene family may be paralogs or orthologs: gene with similar sequences from same or different species. See explanations for 7 classes in our dataset: [Kaggle](https://www.kaggle.com/nageshsingh/demystify-dna-sequencing-with-machine-learning)

| Gene family | Class label |
|:---|---|
| G protein coupled receptors | 0 |
| Tyrosine kinase | 1 |
| Tyrosine phosphatase | 2 |
| Synthetase | 3 |
| Synthase | 4 |
| Ion channel | 5 |
| Transcription | 6 |


Run following commands to get data:

In [ ]:
! unzip dna_sequence_classification/data.zip #&& dna_sequence_classification/data.zip

## Requirements

We will run codes with python3 and start [Milvus2.0 (Standalone)](https://milvus.io/docs/v2.0.0/install_standalone-docker.md) & postgres with [docker](https://docs.docker.com/get-docker/). Required python modules are listed in `requirements.txt` to install.

|    Packages    |     Servers    |
| --------------- | -------------- |
| pymilvus==2.0.0rc8 | milvus-2.0.0-rc8|
|    sklearn    |    mysql    |
|   pymysql   |
|     numpy   |
| pickle-mixin |

## Up and Running

### Install Packages


Install the required python packages with requirements.txt.

In [ ]:
! pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org -r dna_sequence_classification/requirements.txt

### Start Milvus

Download and save docker-compose.standalone.yml as docker-compose.yml. Start Milvus2.0 with docker-compose.

In [ ]:
# ! wget https://github.com/milvus-io/milvus/releases/download/v2.0.0-rc8/milvus-standalone-docker-compose.yml -O docker-compose.yml
# ! docker-compose up -d

### Start Mysql

Milvus2.0 does not suppport string for now. Start mysql as docker container to store and recall non-vector attributes of DNA sequences (eg. id, label/class).

In [ ]:
# !docker run -p 3306:3306 -e MYSQL_ROOT_PASSWORD=123456 -d --name mysql mysql:5.7

### Check Status

In [ ]:
# !docker ps

## Code Overview

### Connect to Servers

Connect to servers with hosts & ports. In this case, the docker containers are running on localhost and the default ports.

In [ ]:
from pymilvus import connections, utility, db
import pymysql

conn = connections.connect(host="standalone", port=19530, db_name="bioregistry")
conn = pymysql.connect(host='mariadb', user='mariadb', port=3306, password='password', database='celeritas',local_infile=True)
cursor = conn.cursor()

### Create Collection, Partitions, Index in Milvus

#### 1. Create Collection

Set collection name and dimension value. Create a collection with fields.
- Collection name: dna_seq
- Dimension: 768
- Fields: pk (primary keys), embedding (dna sequence embeddings)

In [ ]:
import time
from pymilvus import utility, Collection, FieldSchema, DataType, CollectionSchema

time.sleep(.1)

collection_name = "dna_seq"
dim = 768

# Drop the previously stored collection for a clear run
if utility.has_collection(collection_name) == True:
    collection = Collection(collection_name)
    collection.drop()

# Set fields & schema
all_fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim)
        #schema.FieldSchema(name="class", dtype=DataType.STRING)
        ]
default_schema = CollectionSchema(fields=all_fields, 
                                  description="DNA recognition: kmers & vectorizer", 
                                  auto_id=False)

# Create collection
DNA_collection = Collection(name=collection_name, data=None, schema=default_schema)

# Check if collection is successfully created
if utility.has_collection(collection_name):
    print(
    "Collection is successfully created: " + collection_name)
else:
    raise Exception("Fail to create collection: " + collection_name)

#### 2. Create Partitions

Create 3 partitions with proper names: human, chimp, dog

In [ ]:
human_partition = DNA_collection.create_partition('human')
chimp_partition = DNA_collection.create_partition('chimp')
dog_partition = DNA_collection.create_partition('dog')

DNA_collection.partitions

#### 3. Set Index

Set index parameters after collection is created. Here index type is IVF_SQ8 and metric type is Inner Product.

In [ ]:
index_params = {
    'index_type': 'IVF_SQ8',
    'params': {'nlist': 512},
    'metric_type': 'IP'
    }

DNA_collection.create_index(field_name="embedding", index_params=index_params)

# Check if index is successfully set
if DNA_collection.has_index():
    print("Index is successfully set for collection " + collection_name)
else:
    raise Exception("Fail to set index for collection " + collection_name)

### Create Table in Mysql

Create a table with collection name in mySQL to store milvus ids (i.e. field "pk") and corresponding labels.

In [ ]:
# Delete previously stored table for a clean run
drop_table = "DROP TABLE IF EXISTS " + collection_name + ";"
cursor.execute(drop_table)

try:
    sql = "CREATE TABLE if not exists " + collection_name + " (pk TEXT, label TEXT);"
    cursor.execute(sql)
    print("create MySQL table successfully!")
except Exception as e:
    print("can't create a MySQL table: ", e)

### Process & Store Datasets

#### 1. Get Data

Read data from text files as dataframes. Rebuild data and replace original columns with:
- sequence --> subsequences by [k-mer](https://en.wikipedia.org/wiki/K-mer#:~:text=Usually%2C%20the%20term%20k%2Dmer,total%20possible%20k%2Dmers%2C%20where) (k=5)
- class --> label declaring organism & class (e.g. human: 0)

In [ ]:
import numpy as np
import pandas as pd

# Function to get k-mers for sequence s
def build_kmers(s, k):
    kmers = []
    n = len(s) - k + 1

    for i in range(n):
        kmer = s[i : i+k].upper()
        kmers.append(kmer)

    return kmers

# Function to replace sequence column with kmers in df
def seq_to_kmers(df):
    df['kmers'] = df.apply(lambda x: build_kmers(x['sequence'], 4), axis =1)
    df = df.drop(['sequence'],axis=1)


# Read files
human = pd.read_table('./data/human_data.txt')
human = human.sample(frac=1, random_state=2021).reset_index(drop=True)
chimp = pd.read_table('./data/chimp_data.txt')
dog = pd.read_table('./data/dog_data.txt')

# Replace classes with labels (organism: class)
human['label']=['human: ' + str(x) for x in human['class']]
human = human.drop(['class'], axis=1)
chimp['label']=['chimp: ' + str(x) for x in chimp['class']]
chimp = chimp.drop(['class'], axis=1)
dog['label']=['dog: ' + str(x) for x in dog['class']]
dog = dog.drop(['class'], axis=1)

seq_to_kmers(human)
seq_to_kmers(chimp)
seq_to_kmers(dog)

# Combine all dataframes
#df = human.append(chimp).append(dog)
#df = df.sample(frac=1,random_state=1)
#seq_to_kmers(df)

print(human.head())
#print(chimp.head())
#print(dog.head())

Get lists of texts for DNA sequences in k-mers & labels. Split 20 human data to test search performance.

In [ ]:
# Get lists of sequences in k-mers and labels in text from dataframe
def mydata(df):
    texts = []
    labels = []
    words = list(df['kmers']) # list of all sequences in kmers

    for i in range(len(words)):
        texts.append(' '.join(words[i])) 
    
    for x in df['label']:
        labels.append(x)

    if len(texts)!=len(labels):
        raise Exception("Texts & labels length are not equal!")
        
    return (texts, labels)
    
human_texts, human_labels = mydata(human)
chimp_texts, chimp_labels = mydata(chimp)
dog_texts, dog_labels = mydata(dog)

# Split human data to test search performance
test_texts = human_texts[-20:]
actual_labels = human_labels[-20:]
human_texts = human_texts[:-20]
human_labels = human_labels[:-20]

train_texts = human_texts + chimp_texts + dog_texts
train_labels = human_labels + chimp_labels + dog_labels

print("train row count:", len(train_texts))
print("human({})".format(str(len(human_texts)))
      +" chimp({})".format(str(len(chimp_texts)))
      +" dog({})".format(str(len(dog_texts))))
print("test row count:", len(test_texts))

#### 2. Generate Embeddings

Extract features for DNA sequences (after k-mers) by `CountVectorizer` with previously declared dimension. Normalize output by `sklearn.preprocessing` to get final embeddings.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

# Transform sequences in kmers to vectors
def char_to_vec(v_model, text):
    V = v_model.transform(text).toarray()
    #features = vectorizer.get_feature_names()
    embeddings = preprocessing.normalize(V)
    return embeddings

# Train vectorizer model 
vectorizer = CountVectorizer(ngram_range=(4,4), max_features=dim)
X = vectorizer.fit_transform(train_texts).toarray()
train_emb = list(preprocessing.normalize(X))
# print(vectorizer.get_feature_names())

human_emb = train_emb[:len(human_texts)]
chimp_emb = train_emb[len(human_texts):(len(human_texts)+len(chimp_texts))]
dog_emb = train_emb[(len(human_texts)+len(chimp_texts)):len(train_texts)]

In [ ]:
DNA_collection.num_entities 

#### 3. Insert data

##### Insert to Milvus

Insert all embeddings to corresponding partitions with proper primary keys. Don't insert if there exists previous data in collection.

In [ ]:
human_pk = [x for x in range(len(human_emb))]
chimp_pk = [x for x in range(len(human_emb), len(human_emb)+len(chimp_emb))]
dog_pk = [x for x in range(len(human_emb)+len(chimp_emb), len(train_emb))]

if DNA_collection.num_entities == 0:
    DNA_human = DNA_collection.upsert([human_pk, human_emb], partition_name='human')
    DNA_chimp = DNA_collection.upsert([chimp_pk, chimp_emb], partition_name='chimp')
    DNA_dog = DNA_collection.upsert([dog_pk, dog_emb], partition_name='dog')

    DNA_collection.compact()

    if DNA_collection.is_empty:
        print("Insert collection failed.")
    else:
        print(DNA_collection.partitions)
else:
    print("Previous data in this collection!")

##### Insert to Mysql

Insert primary keys in Milvus and corresponding labels into Mysql.

In [ ]:
import os 

# Combine pk and label into a list
def format_data(pk, label):
    data = []
    for i in range(len(pk)):
        value = (str(pk[i]), label[i])
        data.append(value)
    return data

def load_data_to_mysql(cursor, conn, table_name, data):
    sql = "insert into " + table_name + " (pk,label) values (%s,%s);"
    try:
        cursor.executemany(sql, data)
        conn.commit()
        print("MYSQL loads data to table: {} successfully".format(table_name))
    except Exception as e:
        print("MYSQL ERROR: {} with sql: {}".format(e, sql))

all_pk = human_pk + chimp_pk + dog_pk
load_data_to_mysql(cursor, conn, collection_name, format_data(all_pk, train_labels))

### Search

Load collection. Set search parameters with Inner Product as metric_type and nprobe of 20.

In [ ]:
DNA_collection.load()
search_params = {"metric_type": "IP", "params": {"nprobe": 20}}

#### 1. Classify DNA Sequences

The aim is to classify 20 human DNA sequences with labels. Inputs are pre-processed subsequences in text by k-mers (k=4).

In [ ]:
pd.DataFrame(test_texts).head(5)

Transform each input to vector with pre-trained vectorizer model.

In [ ]:
def get_vector(text, vectorizer):
    x = vectorizer.transform(text).toarray()
    return list(preprocessing.normalize(x))

embs = get_vector(test_texts, vectorizer)
test_emb = []
for x in embs:
    test_emb.append(list(x))

Search for top 10 results in human partition for each input vector

In [ ]:
start_time = time.time()
print(f"\nSearch...")
# define output_fields of search result
res = DNA_collection.search(test_emb, "embedding", search_params,
                                    limit=10, partition_names=['human'])
end_time = time.time()
print("search latency = %.4fs" % (end_time - start_time))

Display search result (recall labels from mysql by result ids)

In [ ]:
def get_label_by_pk(cursor, m_pk, table_name):
    sql = "select label from " + table_name + " where pk=" + str(m_pk) +";"
    try:
        cursor.execute(sql)
        myresult = cursor.fetchall()
        myresult = [x[0] for x in myresult]
        return myresult
    except Exception as e:
        print("MYSQL ERROR: {} with sql: {}".format(e, sql))
        
for i in range(len(res)):
    print("\nSearch for '{}'".format(actual_labels[i]))
    print('[label]', 'distance')
    for x in res[i]:
        C = get_label_by_pk(cursor, str(x.id), collection_name)
        D = x.distance
        print(C, D)

#### 2. Compare Similarity

According to IP distance value got from similarity search in Milvus, we can compare similarities between organisms.

Search top 1 results for 800 chimp/dog vectors in human partition. Average distances to reflect how close between chimp/dog and human DNA sequences. The larger the IP distance, the closer between organisims with respect to DNA sequence.

In [ ]:
chimp_search = []
dog_search = []
for x in chimp_emb[:800]:
    chimp_search.append(list(x))
for x in dog_emb[:800]:
    dog_search.append(list(x))

chimp_res = DNA_collection.search(chimp_search, "embedding", search_params,
                                    limit=1, partition_names=['human'])
dog_res = DNA_collection.search(dog_search, "embedding", search_params,
                                    limit=1, partition_names=['human'])

def similarity(search_res):
    total_d = 0
    for hits in search_res:
        total_d = total_d + sum(hits.distances)/len(hits)
    return total_d/(len(search_res))

print('chimp-human similarity score:', similarity(chimp_res))
print('dog-human similarity score:', similarity(dog_res))

From above similarity scores from milvus, we can tell that chimpanzee is closer to human compared to dog, fitting the [biological research facts](https://education.seattlepi.com/animals-share-human-dna-sequences-6693.html) "...humans share 98.8 percent of their DNA with bonobos and chimpanzees...Humans and dogs share 84 percent of their DNA"